## Lasso Regression
This notebook runs the drug repurposing project's features for feature selection using the properties in lasso regression. It takes the training set from the project with 114 features reduced from R, and run Lasso for 5, 10, 15, 20, 25, 30, 40, 60, 80, 114 features to see which is the optimal numbers of features for building the predictive models.

### Predictive models

In [ ]:
import random
import math

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,LabelBinarizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score, RocCurveDisplay, PrecisionRecallDisplay, ConfusionMatrixDisplay
from sklearn.linear_model import Lasso

import pymc as pm
import arviz as az

from collections import Counter
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

# Set working directory
path = '/content/drive/My Drive/Tox21_data/'

Mounted at /content/drive


read stacked X and y

In [ ]:
X = pd.read_csv(path + '114_fea_resampled_X.csv')
X
y = pd.read_csv(path + 'stacked_resampled_y.csv')
y = y.drop(y.columns[0],axis=1)
y

,Outcome
0,0
1,0
2,0
3,0
4,0
...,...
57685,1
57686,1
57687,1
57688,1


In [ ]:
y.Outcome.value_counts()

0    28845
1    28845
Name: Outcome, dtype: int64

In [ ]:
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, random_state = 42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
logistic_cv_balanced = LogisticRegressionCV(penalty='l2',Cs=10,
                                            max_iter=10000,class_weight=None).fit(X_train, y_train)

coef = logistic_cv_balanced.coef_[0]

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
y_train_pred = logistic_cv_balanced.predict(X_train)
y_pred = logistic_cv_balanced.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75      5822
           1       0.75      0.75      0.75      5716

    accuracy                           0.75     11538
   macro avg       0.75      0.75      0.75     11538
weighted avg       0.75      0.75      0.75     11538

              precision    recall  f1-score   support

           0       0.75      0.76      0.76     23023
           1       0.76      0.75      0.76     23129

    accuracy                           0.76     46152
   macro avg       0.76      0.76      0.76     46152
weighted avg       0.76      0.76      0.76     46152



In [ ]:
abs_coef = abs(coef)
abs_coef = pd.DataFrame(abs_coef, columns=['abs_coef'])
abs_coef['features'] = X.columns
abs_coef

,abs_coef,features
0,0.151006,MinEStateIndex
1,0.128285,MaxAbsEStateIndex
2,0.135245,MinAbsEStateIndex
3,0.107575,qed
4,0.294596,MaxPartialCharge
...,...,...
109,0.136814,fr_methoxy
110,0.012180,fr_para_hydroxylation
111,0.380965,fr_phenol_noOrthoHbond
112,0.026601,fr_piperdine


In [ ]:
abs_coef = abs_coef.sort_values(by=['abs_coef'], ascending=False)
abs_coef

,abs_coef,features
92,1.023393,fr_C_O
58,0.919649,TPSA
41,0.781931,SMR_VSA1
84,0.700246,NumHAcceptors
82,0.688774,NumAromaticCarbocycles
...,...,...
100,0.015092,fr_aniline
110,0.012180,fr_para_hydroxylation
65,0.011287,EState_VSA6
66,0.000340,EState_VSA7


In [ ]:
fea_rank = abs_coef.features.tolist()
fea_rank[:40]

['fr_C_O',
 'TPSA',
 'SMR_VSA1',
 'NumHAcceptors',
 'NumAromaticCarbocycles',
 'NumHeteroatoms',
 'Chi4n',
 'VSA_EState10',
 'VSA_EState3',
 'Kappa1',
 'Chi2v',
 'SMR_VSA10',
 'SlogP_VSA2',
 'HallKierAlpha',
 'SMR_VSA5',
 'VSA_EState7',
 'FractionCSP3',
 'SlogP_VSA10',
 'BCUT2D_LOGPLOW',
 'VSA_EState2',
 'FpDensityMorgan2',
 'fr_phenol_noOrthoHbond',
 'SlogP_VSA1',
 'PEOE_VSA1',
 'fr_Ar_N',
 'BCUT2D_CHGLO',
 'fr_ether',
 'EState_VSA1',
 'MinPartialCharge',
 'VSA_EState6',
 'MaxPartialCharge',
 'MinAbsPartialCharge',
 'VSA_EState9',
 'SMR_VSA3',
 'FpDensityMorgan3',
 'SlogP_VSA12',
 'fr_Al_OH_noTert',
 'BalabanJ',
 'MaxAbsPartialCharge',
 'EState_VSA10']

### Run lasso

In [ ]:
logistic_lasso = LogisticRegressionCV(penalty='l1',class_weight='balanced', solver = 'saga').fit(X_train, y_train)
coef_w = logistic_lasso.coef_[0]

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packag

In [ ]:
y_pred = logistic_lasso.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75      5822
           1       0.75      0.75      0.75      5716

    accuracy                           0.75     11538
   macro avg       0.75      0.75      0.75     11538
weighted avg       0.75      0.75      0.75     11538



In [ ]:
abs_coef_lasso = abs(coef_w)
abs_coef_lasso = pd.DataFrame(abs_coef_lasso, columns=['abs_coef'])
abs_coef_lasso['features'] = X.columns
abs_coef_lasso = abs_coef_lasso.sort_values(by=['abs_coef'], ascending=False)
abs_coef_lasso

,abs_coef,features
92,0.906455,fr_C_O
58,0.677109,TPSA
82,0.562902,NumAromaticCarbocycles
41,0.561535,SMR_VSA1
84,0.538694,NumHAcceptors
...,...,...
18,0.011235,BCUT2D_MRLOW
62,0.007032,EState_VSA3
6,0.006141,MaxAbsPartialCharge
54,0.004353,SlogP_VSA4


In [ ]:
coef_lasso_fea = abs_coef_lasso.features.tolist()
coef_lasso_fea[:40]

['fr_C_O',
 'TPSA',
 'NumAromaticCarbocycles',
 'SMR_VSA1',
 'NumHAcceptors',
 'VSA_EState10',
 'SlogP_VSA10',
 'Chi2v',
 'VSA_EState3',
 'fr_phenol_noOrthoHbond',
 'Chi4n',
 'FractionCSP3',
 'Kappa1',
 'BCUT2D_LOGPLOW',
 'fr_Ar_N',
 'NumHeteroatoms',
 'VSA_EState7',
 'BCUT2D_CHGLO',
 'SMR_VSA10',
 'SMR_VSA5',
 'HallKierAlpha',
 'fr_ether',
 'PEOE_VSA1',
 'SMR_VSA3',
 'SlogP_VSA2',
 'EState_VSA1',
 'MolLogP',
 'fr_C_O_noCOO',
 'VSA_EState9',
 'SlogP_VSA12',
 'FpDensityMorgan2',
 'fr_ester',
 'SlogP_VSA1',
 'BalabanJ',
 'EState_VSA10',
 'BCUT2D_MRHI',
 'PEOE_VSA14',
 'VSA_EState6',
 'BCUT2D_CHGHI',
 'SMR_VSA4']

In [ ]:
pd.DataFrame(coef_lasso_fea).to_csv(path+ '114_feature_ranked_by_lasso.csv', index=False)

Test for the optimal numbers of features

In [ ]:
y_pred_r = []
for i in [5, 10, 15, 20, 25, 30, 40, 60, 80, 100, 114]:
  num_fea = coef_lasso_fea[:i]
  X_reduce = X[X.columns.intersection(num_fea)]
  scaler = StandardScaler()
  X_train, X_test, y_train, y_test = train_test_split(X_reduce, y, test_size=0.20, shuffle=True, random_state = 42)
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  logistic_cv_balanced = LogisticRegressionCV(penalty='l1', solver = 'saga').fit(X_train, y_train)
  # logistic_cv_balanced = LogisticRegressionCV(penalty='l2',Cs=10,
  #                                             max_iter=10000,class_weight=None).fit(X_train, y_train)
  y_pred = logistic_cv_balanced.predict(X_test)
  y_pred_r.append(y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [ ]:
bal_acc = []
for i in range(len(y_pred_r)):
  print(balanced_accuracy_score(y_test,y_pred_r[i]))
  bal_acc.append(balanced_accuracy_score(y_test,y_pred_r[i]))

0.6430375336042266
0.7094061063714551
0.7185717395396289
0.7303455390727337
0.7330191229474168
0.736233355345509
0.7394936534498255
0.749688267686647
0.747900569712288
0.7494942087624485
0.7515903937166497


In [ ]:
bal_acc_df = pd.DataFrame(bal_acc, columns=['bal_acc'])
bal_acc_df['#fea'] = [5, 10, 15, 20, 25, 30, 40, 60, 80, 100, 114]
bal_acc_df

,bal_acc,#fea
0,0.643038,5
1,0.709406,10
2,0.718572,15
3,0.730346,20
4,0.733019,25
5,0.736233,30
6,0.739494,40
7,0.749688,60
8,0.747901,80
9,0.749494,100


### Check to see if the stacked data is the same with stratified data

In [ ]:
label_encoder = preprocessing.LabelEncoder()
resampled_data_X = []
resampled_data_y = []
feature_top_40 = ['FractionCSP3', 'VSA_EState3', 'SlogP_VSA10', 'PEOE_VSA14', 'fr_Al_COO',
                  'MolLogP', 'SlogP_VSA3', 'VSA_EState9', 'VSA_EState7', 'BCUT2D_LOGPLOW',
                  'qed', 'MinAbsEStateIndex', 'SlogP_VSA1', 'VSA_EState10',
                  'FpDensityMorgan3', 'fr_methoxy', 'MaxAbsPartialCharge', 'BCUT2D_MWHI',
                  'fr_ester', 'BCUT2D_MRHI', 'fr_NH1', 'fr_C_O_noCOO',
                  'MinAbsPartialCharge', 'MinPartialCharge', 'VSA_EState5', 'TPSA',
                  'SlogP_VSA8', 'SlogP_VSA6', 'fr_NH0', 'PEOE_VSA10', 'SlogP_VSA2',
                  'fr_Al_OH_noTert', 'BalabanJ', 'Chi4v', 'VSA_EState1', 'VSA_EState4',
                  'SMR_VSA4', 'fr_Ar_N', 'EState_VSA8', 'BCUT2D_CHGLO']
for i in range(len(df)):
  one_assay = df[i]
  one_assay = one_assay.drop(one_assay.columns[0],axis=1).drop_duplicates(subset = 'SMILES')
  y = label_encoder.fit_transform(one_assay.iloc[:,0])
  # X = one_assay[one_assay.columns.intersection(feature_top_40)]
  X = one_assay.iloc[:,2:]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, random_state = 42)
  near_miss = RandomUnderSampler(random_state = 42)  #NearMiss(version = 1, n_neighbors = 3)
  X_train_resampled, y_train_resampled = near_miss.fit_resample(X_train,y_train)
  resampled_data_X.append(X_train_resampled)
  resampled_data_y.append(y_train_resampled)

In [ ]:
resampled_data_X[0]

In [ ]:
pd.concat(resampled_data_X).reset_index(drop=True).to_csv('drive/My Drive/Tox21_data/stacked_resampled_X.csv')

stack_y = []
for i in range(50):
  df = pd.DataFrame(resampled_data_y[i])
  stack_y.append(df)

pd.concat(stack_y).reset_index(drop=True).rename(columns = {0: 'Outcome'}).to_csv('drive/My Drive/Tox21_data/stacked_resampled_y.csv')

In [ ]:
df_list = []
for i in range(len(df)):
  one_assay = df[i]
  one_assay = one_assay.drop(one_assay.columns[0], axis=1).drop_duplicates(subset=['SMILES'])
  one_assay['ProtocolName'] = one_assay.columns[0]
  one_assay = one_assay.rename(columns = {one_assay.columns[0]: 'Outcome'})
  X = one_assay[one_assay.columns.intersection(feature_top_40)]
  X['ProtocolName'] = one_assay.iloc[:,-1]
  X['Outcome'] = one_assay.Outcome
  df_list.append(X)


df_stack_50 = pd.concat(df_list).reset_index(drop=True)

In [ ]:
X = df_stack_50.iloc[:,:-1]
y = df_stack_50.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, random_state = 42, stratify=X['ProtocolName'])
near_miss = RandomUnderSampler(random_state = 42)  #NearMiss(version = 1, n_neighbors = 3)
X_train_resampled, y_train_resampled = near_miss.fit_resample(X_train,y_train)

In [ ]:
X_train_resampled